In [40]:
from pathlib import Path
import csv
import sys
from feat import Detector
import logging
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor

import os

In [41]:
which_algorithm = 'xgb'
cores = os.cpu_count() - 1
loc_or_serv = 'local'

In [42]:
path_to_save_local = '/Users/zeleninam2/Documents/1_projects/1_FACE_PAIN/proj_fex_software_comparison/outputs/'
path_to_save_server = '/Volumes/LYA/LYA_Lab/FEX_Substudy1/BIDS_dataset/code/FEX_AU_comparison_Marie/FEX_AU_outputs/'

In [43]:
file_all_folders_server = '/Users/zeleninam2/Documents/1_projects/1_FACE_PAIN/proj_fex_software_comparison/mycode/AU_comparison_software/run_libreface/all_folders.txt'
file_all_folders_local = '/Users/zeleninam2/Documents/1_projects/1_FACE_PAIN/proj_fex_software_comparison/mydata/test_data/all_folders_local.txt'

In [44]:
if loc_or_serv == 'local':
    path_to_save = path_to_save_local
    file_all_folders = file_all_folders_local
else:
    path_to_save = path_to_save_server
    file_all_folders = file_all_folders_server  

In [45]:
log_path = f"{path_to_save}/mylog_pyfeat_{which_algorithm}.log"

In [46]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        #logging.StreamHandler(sys.stdout),  # console
        logging.FileHandler(log_path, mode="w", encoding="utf-8"),  # file
    ]
)

In [49]:
def log(msg): # helper function
    logging.info(msg)  
    print(msg)

def process_file(path_to_file):
    filename_stem = path_to_file.stem
    filename = path_to_file.name                   
    log(f"---------------------------------------------------------------- \n Processing file {filename} \n ----------------------------------------------------------------")
    # for this file:                  
    # Do pyfeat processing              
    log(f"\n Doing pyfeat processing...")        
    video_prediction = detector.detect_video(str(path_to_file), data_type="video")        
    log(f"\n Pyfeat processing: DONE")                                 
    total_frames_vid = video_prediction.shape[0]
    processed_frames_vid = video_prediction.FaceScore.isna().value_counts()[False]  
    return video_prediction, filename, total_frames_vid, processed_frames_vid
    
try:
    log(f'\n\n==============================\n START PYFEAT ({which_algorithm.upper()}) PROCESSING \n=============================\n')   
    log('\nPrep work...')
            
    if which_algorithm == 'xgb':
        # define paths
        my_path_to_save = path_to_save + 'outputs_pyfeat_xgb'
        # load pyfeat detector module
        detector = Detector(au_model='xgb')
            
    elif which_algorithm == 'svm':
        # define paths
        my_path_to_save = path_to_save + 'outputs_pyfeat_svm'
        # load pyfeat detector module
        detector = Detector(au_model='svm')    
    
    # prep file to save frames data
    frames_file = f"{my_path_to_save}/frames_pyfeat_{which_algorithm}.csv"
    with open(frames_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            f"name_pf_{which_algorithm}",
            f"total_frames_pf_{which_algorithm}",
            f"processed_frames_pf_{which_algorithm}",])
        
    log('\nPrep work DONE\n')

    # read all folders data, iterate
    with open(file_all_folders) as f:
        for line in f:
    
            # work with data folder by folder:                
                
            folderpath = Path(line.strip())
            name_base = (Path(folderpath).name)
    
            files_in_dir = [
                p for p in Path(folderpath).iterdir()
                if p.is_file() and p.suffix.lower() == ".mp4"
            ]
                
            howmanyvidsindir = len(files_in_dir)
    
            log("\n+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
            log(f"+++++++++++++++++++++++++++++++++++++++ Processing FOLDER: {name_base} +++++++++++++++++++++++++++++++++++++++")
            log(f"+++++++++++++++++++++++++++++++++++++++ Total videos: {howmanyvidsindir} +++++++++++++++++++++++++++++++++++++++++++++++++++")
            log("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")

            with ThreadPoolExecutor(max_workers=how_many_cores) as executor:
                pyfeat_results = list(executor.map(process_file, files_in_dir))
            print(pyfeat_results)
                            
except KeyboardInterrupt:
    log("Script interrupted by user!")

except Exception as e:
    logging.exception(f"Script crashed: {e}")  # logs exception with traceback
    
finally:
    logging.shutdown()  # flush everything immediately




 START PYFEAT (XGB) PROCESSING 


Prep work...

Prep work DONE


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++ Processing FOLDER: sub-9473 +++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++ Total videos: 5 +++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++



In [37]:
pyfeat_results

NameError: name 'pyfeat_results' is not defined